# Programmering, gör din egna självinlärda artificiella intelligens

In [1]:
# BlackJack - By Rickard Maus

import random
import numpy as np

deck = {"ace1" : 1, "ace2" : 1, "ace3" : 1, "ace4" : 1, "two1" : 2, "two2" : 2, "two3" : 2, "two4" : 2,
       "thr1" : 3, "thr2" : 3, "thr3" : 3, "thr4" : 3, "for1" : 4, "for2" : 4, "for3" : 4, "for4" : 4,
       "fiv1" : 5, "fiv2" : 5, "fiv3" : 5, "fiv4" : 5, "six1" : 6, "six2" : 6, "six3" : 6, "six4" : 6,
       "sev1" : 7, "sev2" : 7, "sev3" : 7, "sev4" : 7, "eit1" : 8, "eit2" : 8, "eit3" : 8, "eit4" : 8,
       "nin1" : 9, "nin2" : 9, "nin3" : 9, "nin4" : 9, "ten1" : 10, "ten2" : 10, "ten3" : 10, "ten4" : 10,
       "J1" : 10, "J2" : 10, "J3" : 10, "J4" : 10, "Q1" : 10, "Q2" : 10, "Q3" : 10, "Q4" : 10,
       "K1" : 10, "K2" : 10, "K3" : 10, "K4" : 10}

class b_jack():
    def __init__(self):
        self.deck_L = list(deck)
        self.Player_hand = []
        self.Dealer_hand = []
        self.hold = np.array([])
        self.Player_hand_size = 0
        self.Dealer_hand_size = 0
        self.reward = 0
        self.pull_c = 2
        self.n_Player = 2
        self.bustvar = False
        self.tie = False
        self.arr = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(1,12)
        
    def arr_start_assign(self):
        self.arr[0,0] = deck[self.Player_hand[0]]
        self.arr[0,1] = deck[self.Player_hand[1]]
        
    def player_hand_count(self):
        P_hand_s = len(self.Player_hand)
        self.Player_hand_size = 0
        for n in range(0, P_hand_s):
            self.Player_hand_size = self.Player_hand_size + deck[self.Player_hand[n]]
    
    def dealer_hand_count(self):
        D_hand_s = len(self.Dealer_hand)
        self.Dealer_hand_size = 0
        for n in range(0, D_hand_s):
            self.Dealer_hand_size = self.Dealer_hand_size + deck[self.Dealer_hand[n]]
        
    def bust(self, x):
        Bust = 0
        hand_s = len(x)
        for n in range(0, hand_s):
            Bust = Bust + deck[x[n]]
        if (Bust > 21) and (x == self.Player_hand):
            self.reward = -1
        elif (Bust > 21) and (x == self.Dealer_hand):
            self.bustvar = True
            self.reward = 1
    
    def deal(self, x):
        draw = ''.join(random.sample(self.deck_L,1))
        self.deck_L.remove(draw)
        if x%2 == 1:
            self.Player_hand.append(draw)
        else:
            self.Dealer_hand.append(draw)
            
    def game_start(self):
        for x in range(0,4):
            self.deal(x)
        self.arr_start_assign()
        self.player_hand_count()

    def mid_game(self, x):
        if x == 0:
            self.reward = 0
            self.deal(1)
            self.player_hand_count()
            self.bust(self.Player_hand)
            self.arr[0,self.pull_c] = deck[self.Player_hand[self.n_Player]]
            self.pull_c += 1
            self.n_Player += 1
        else:
            self.end_game()
        return self.arr, self.reward
            
    def end_game(self):
        for x in range(0,12):
            self.dealer_hand_count()
            self.player_hand_count()
            self.bust(self.Dealer_hand)
            if (self.bustvar == True):
                break
            elif (self.Dealer_hand_size > self.Player_hand_size):
                self.reward = -1
                break
            elif (self.Dealer_hand_size == self.Player_hand_size):
                self.tie = True
                self.reward = -1
                break
            elif (self.Dealer_hand_size < 17):
                self.deal(0)
            else:
                self.reward = 1
                break
            

In [2]:
import tensorflow as tf
import time
learning_rate=0.0000007

graph = tf.Graph()
with graph.as_default():
    tf_hand = tf.placeholder(tf.float32, shape=(1,12))
    nextQ = tf.placeholder(tf.float32, shape=[1,2])
    
    weights = {
        'W1' : tf.Variable(tf.random_uniform([12,12],0,1))
        ,'W2' : tf.Variable(tf.random_uniform([12,2],0,1))
    }
    
    def Q_model(x):
        layer_1 = tf.matmul(x, weights['W1'])
        layer_out = tf.matmul(tf.nn.relu(layer_1), weights['W2'])
        return layer_out
    

    loss = tf.reduce_sum(tf.square(nextQ - Q_model(tf_hand)))
    trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    updateModel = trainer.minimize(loss)
    
    Q_values = Q_model(tf_hand)
    predict = tf.argmax(Q_values,1)
    

In [ ]:
y = 0.9
num_episodes = 1000000
e = 0.5
e_lower_limit = 0.01
annealing = (e-e_lower_limit)/num_episodes
actions = [0, 1]
tot_win = 0
win = 0
lose = 0
losstotavrg = 0
winrate_list = []
loserate_list = []
Loss_list = []
list_draw_statistics = []
draw_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
point_total_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
miniepoch = 25000

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for i in range(num_episodes):
        game = b_jack()
        game.game_start()
        s = game.arr
        hey = False
        Losstot = 0
        if (i % miniepoch == 0):
            print(s)
        for j in range(1,13):
            prev = None
            game.hold,allQ = sess.run([predict,Q_values], feed_dict={tf_hand:s})
            if np.random.rand(1) < e:
                prev = game.hold
                game.hold[0] = random.sample(actions,1)[0]
            point_total_statistics[game.Player_hand_size-2] += 1
            if game.hold[0] == 0:
                draw_statistics[game.Player_hand_size-2] += 1
            s1,r = game.mid_game(game.hold[0])
            if (i % miniepoch == 0):
                print(allQ)
                print(game.hold, prev)
                print(game.Dealer_hand, "Dealer hand")
                print(r, " r")
                print(s1)
                hey = True
                print("Winrate: {}%".format(100*win/miniepoch))
            Q1 = sess.run(Q_values,feed_dict={tf_hand:s1})
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,game.hold[0]] = r + y*maxQ1
            _,L,w1,w2 = sess.run([updateModel,loss,weights['W1'],weights['W2']],feed_dict={tf_hand:s,nextQ:targetQ})
            Losstot += L
            s = s1
            if (r == 1) or game.tie:
                e = e-annealing
                if r == 1:
                    win += 1
                    tot_win += 1
                break
            elif (r == -1):
                lose += 1
                break
        losstotavrg += (Losstot/j)
        if hey == True and (i != 0):
            for n in range(0,20):
                draw_statistics[n] = (draw_statistics[n]/point_total_statistics[n])*100
            print(draw_statistics)
            print(e)
            list_draw_statistics.append(draw_statistics)
            draw_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            point_total_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            Loss_list.append((losstotavrg/miniepoch))
            winrate_list.append((win/miniepoch)*100)
            loserate_list.append((lose/miniepoch)*100)
            losstotavrg = 0
            win = 0
            lose = 0
            print("---------------------")
        elif hey == True:
            print("---------------------")
print(Loss_list)
print(winrate_list)
print(loserate_list)
print("Wins:", win)
print("Losses", lose)
print("Winrate after {} games lr={}: {}%".format(num_episodes, learning_rate, 100*win_tot/num_episodes))
            

[[8 3 0 0 0 0 0 0 0 0 0 0]]
[[46.196823 34.18092 ]]
[1] [1]
['fiv2', 'thr3', 'ace1', 'fiv1'] Dealer hand
-1  r
[[8 3 0 0 0 0 0 0 0 0 0 0]]
Winrate: 0.0%
---------------------
[[10  2  0  0  0  0  0  0  0  0  0  0]]
[[47.89381 35.72805]]
[0] [0]
['two1', 'J2'] Dealer hand
0  r
[[10  2  4  0  0  0  0  0  0  0  0  0]]
Winrate: 10.952%
[[61.97152  44.951374]]
[0] [0]
['two1', 'J2'] Dealer hand
-1  r
[[10  2  4  8  0  0  0  0  0  0  0  0]]
Winrate: 10.952%
[73.38709677419355, 74.44794952681389, 75.97402597402598, 73.51724137931035, 76.20221948212084, 76.83168316831683, 73.56608478802993, 75.63758389261746, 74.06969875959834, 75.3541076487252, 76.91778465772275, 75.24038461538461, 74.20675537359263, 75.88652482269504, 74.15268743580965, 74.08487204405571, 75.05140507196711, 74.11323328785812, 75.02408477842005, 74.59847844463229]
0.498053719999922
---------------------
[[10  7  0  0  0  0  0  0  0  0  0  0]]
[[60.356335 44.151054]]
[1] [1]
['ten4', 'K4'] Dealer hand
-1  r
[[10  7  0  0  0  0

[[ 9 10  0  0  0  0  0  0  0  0  0  0]]
[[18.514402 15.102948]]
[1] [1]
['two4', 'ace2', 'ten2', 'K2'] Dealer hand
1  r
[[ 9 10  0  0  0  0  0  0  0  0  0  0]]
Winrate: 10.284%
[73.94957983193278, 74.0625, 81.20805369127517, 75.53846153846155, 75.05910165484634, 77.4904214559387, 78.1387181738367, 75.86206896551724, 74.47447447447448, 76.87004481213374, 76.05281941470379, 76.22237106496986, 76.65995975855131, 75.91167614586819, 76.54569892473118, 75.47169811320755, 75.88176016123614, 76.24419376244194, 77.29973853101973, 76.42069550466498]
0.4740221599989587
---------------------
[[10 10  0  0  0  0  0  0  0  0  0  0]]
[[16.3204   13.970347]]
[0] [0]
['eit2', 'six2'] Dealer hand
-1  r
[[10 10 10  0  0  0  0  0  0  0  0  0]]
Winrate: 10.504%
[83.47107438016529, 78.51851851851852, 78.04878048780488, 76.74772036474164, 75.80853816300129, 74.92566897918731, 75.41390728476821, 76.28865979381443, 75.54086538461539, 75.59972583961617, 75.58505064617535, 76.80081163341225, 74.84787018255578, 7